# Introduction

Project Euler is about using math and computer science principals to solve problems.

# Problem 1: Multiples of 3 and 5

If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6, and 9. The sum of these multiples is 23.

Find the sum of all multiples of 3 or 5 below 1000.

(Answer is 233168)



In [31]:
def three_or_five_1(n):
  return sum([i for i in range(n) if i%3 == 0 or i%5 == 0])
                    
def three_or_five_2(n):
  return sum(set(range(0, n, 3)).union(set(range(0, n, 5))))

# A third solution exists which involves double-counting, then subtracting
def sum_all_m(n, m):
  return sum(range(0, n, m))
# sum_all(1000, 3) + sum_all(1000, 5) - sum_all(1000, 15)
               

In [32]:
print(three_or_five_1(1000))
print(three_or_five_2(1000))
print(sum_all_m(1000, 3) + sum_all_m(1000, 5) - sum_all_m(1000, 15))

233168
233168
233168


In [33]:
%%timeit
three_or_five_1(1000)

111 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [34]:
%%timeit
three_or_five_2(1000)

24.5 µs ± 186 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [35]:
%%timeit
sum_all_m(1000, 3) + sum_all_m(1000, 5) - sum_all_m(1000, 15)

7.76 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Problem 2: Even Fibonacci Numbers
Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with 1 and 2, the first 10 terms will be:

$1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...$

By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.

For the uninitiated, the fibonacci sequence is generated by starting with zero, then adding 1, then....:

$$
n_{0} = 0
$$

$$
n_{1} = n_{0} + 1 \equiv 1
$$

$$
...
n_{n} = n_{n-1} + n_{n-2}
$$

Answer: 4613732


In [108]:
# Solve with generators
def fib(start = 0, acc = 1):
  yield start + acc
  yield from fib(acc, start + acc)


In [109]:
def sum_even_fib_less_than_n(g, n):
  """
  Sum all even fibonacci numbers less than n.
  Args:
    g: Generate fib sequence generator g
    n: Max value (non-inclusive) to sum
  Returns: the sum
  """
  sum_numbers = 0
  while True:
    next_number = next(g)
    if next_number < n:
      if next_number %2 == 0:
        sum_numbers += next_number
    else:
      return sum_numbers

In [110]:
sum_even_fib_less_than_n(g = fib(), n=4000000)

4613732

In [111]:
%%timeit
sum_even_fib_less_than_n(g=fib(), n=4000000)

31.1 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


The solution claims that every third fib number is even, and therefore the most performent solution is the one that doesn't check if every number is even.

In [112]:
g = fib(start=0, acc=1)
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

1
2
3
5
8
13
21
34


In [113]:
def sum_even_fib_less_than_n_2(g, n):
  """
  Sum all even fibonacci numbers less than n. Uses the number rule to avoid 
  checking for even numbers...wild.
  Args:
    g: Generate fib sequence generator g
    n: Max value (non-inclusive) to sum
  Returns: the sum
  """
  next(g)
  sum_numbers = next(g)
  
  while True:
    next(g)
    next(g)
    next_number = next(g)
    if next_number < n:
      sum_numbers += next_number
    else:
      return sum_numbers

In [114]:
sum_even_fib_less_than_n_2(g=fib(), n=4000000)

4613732

In [115]:
%%timeit
sum_even_fib_less_than_n_2(g=fib(), n=4000000)

31.1 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Hmm, no performance improvement. To me this indicates that its actually more about the recursion relationship for 'every third fibonacci number' than it is about the check for whether or not its even.b

# Problem 3 - Largest prime factor
The prime factors of 13195 are 5, 7, 13 and 29.

What is the largest prime factor of the number 600851475143 ?

## Solution

One way to solve this problem is to find all prime numbers less than the number, and then choose the number from this set that has an even divisor. Another method could be to find all factors of the number, and then test if they are prime.

The question is, which method is more performant? Lets do an analysis with simple functions where we look, given an integer, what is the number of primes greater than or equal to that integer, vs the number of factors.

The answer will tell us which one is more important to identify.